<a href="https://colab.research.google.com/github/Darsh0210/Projects/blob/master/NueralNetworks_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [21]:
import pandas as pd

In [5]:
physical_devicess = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devicess[0], True)

In [33]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()
X_train = X_train.reshape(-1,28,28,1).astype("float32")/255.0
X_test  = X_test.reshape(-1,28,28,1).astype("float32")/255.0

In [19]:
y_train[0]

5

In [14]:
# CNN -> BatchNormal -> Relu
class CNNBlock(layers.Layer):
  def __init__(self, output_channels, kernel_size = 3):
    super(CNNBlock, self).__init__()
    self.conv = layers.Conv2D(output_channels, kernel_size, padding='same')
    self.bn  = layers.BatchNormalization()

  def call(self, input_tensor, training = False):
    x = self.conv(input_tensor)
    x = self.bn(x, training = training)
    x = tf.nn.relu(x)
    return x

In [16]:
model = keras.Sequential([
                          CNNBlock(32),
                          CNNBlock(64),
                          CNNBlock(128),
                          layers.Flatten(),
                          layers.Dense(10)
])

In [17]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [18]:
model.fit(X_train, y_train, batch_size = 64, epochs = 3)

Epoch 1/3
938/938 [==============================] - 29s 24ms/step - loss: 1.3688 - accuracy: 0.8955
Epoch 2/3
938/938 [==============================] - 22s 24ms/step - loss: 0.1081 - accuracy: 0.9804
Epoch 3/3
938/938 [==============================] - 22s 24ms/step - loss: 0.0340 - accuracy: 0.9895


In [20]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 6ms/step - loss: 0.0517 - accuracy: 0.9839


[0.051683858036994934, 0.9839000105857849]

In [23]:
y_predicted = model.predict_classes(X_test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [24]:
pd.crosstab(y_test, y_predicted,)

col_0,0,1,2,3,4,5,6,7,8,9
row_0,,,,,,,,,,
0,973,0,3,1,0,1,1,0,1,0
1,0,1104,12,1,2,1,1,0,14,0
2,1,0,1026,0,1,0,1,1,1,1
3,0,0,3,1000,0,4,0,1,2,0
4,0,0,1,0,972,0,1,0,2,6
5,1,0,1,5,0,882,2,0,1,0
6,7,1,1,1,1,8,937,0,2,0
7,0,1,18,1,1,0,0,994,3,10
8,4,0,3,0,1,3,0,1,961,1


ResNetwork Implementation

In [56]:
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding="same")
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x

In [57]:
class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0], 3)
        self.cnn2 = CNNBlock(channels[1], 3)
        self.cnn3 = CNNBlock(channels[2], 3)
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding="same")

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training,)
        x = self.pooling(x)
        return x


In [58]:
class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.classifier(x)
        return x

In [59]:
model = ResNet_Like(num_classes=10)

In [60]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [61]:
model.fit(X_train, y_train, batch_size = 64, epochs = 3)

Epoch 1/3
938/938 [==============================] - 57s 59ms/step - loss: 0.2363 - accuracy: 0.9298
Epoch 2/3
938/938 [==============================] - 55s 58ms/step - loss: 0.0408 - accuracy: 0.9865
Epoch 3/3
938/938 [==============================] - 55s 58ms/step - loss: 0.0285 - accuracy: 0.9916


In [54]:
# model.summary()

938/938 - 36s - loss: 0.1080 - accuracy: 0.9667
157/157 - 2s - loss: 0.0622 - accuracy: 0.9809


[0.06222720816731453, 0.98089998960495]